# PHE SEIR Model Paper results

In this notebook we present how to use the `epimodels` module to reproduce the results in the paper introducing the model built by Public Health England in collaboration with University of Cambridge, using region specific contact matrices from Feb 2020 to June 2020.

In [1]:
# Load necessary libraries
import os
import numpy as np
import pandas as pd
from scipy.stats import gamma, nbinom
import epimodels as em
import matplotlib
import plotly.graph_objects as go
import plotly.express as px
from matplotlib import pyplot as plt
from iteration_utilities import deepflatten

### Define setup matrices for the PHE Model

In [2]:
# Populate the model
total_days =  90
regions = [
    'London', 'Outside London']
age_groups = ['0-1', '1-5', '5-15', '15-25', '25-45', '45-65', '65-75', '75+']

weeks = list(range(1,int(np.ceil(total_days/7))+1))
matrices_region = []

### Variable
# # Initial state of the system
# for w in weeks:
#     weeks_matrices_region = []
#     for r in regions:
#         path = os.path.join('../../data/final_contact_matrices/{}_W{}.csv'.format(r, w))
#         region_data_matrix = pd.read_csv(path, header=None, dtype=np.float64)
#         #region_data_matrix_var.iloc[:, 5] = region_data_matrix_var.iloc[:, 5] * 2
#         regional = em.RegionMatrix(r, age_groups, region_data_matrix)
#         weeks_matrices_region.append(regional)
# 
#     matrices_region.append(weeks_matrices_region)
# 
# contacts = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
# matrices_contact = [contacts]
# 
# # Matrices contact
# time_changes_contact = [1]
# time_changes_region = np.arange(1, total_days+1, 7).tolist()

### Fixed
# Initial state of the system
weeks_matrices_region = []
for r in regions:
    path = os.path.join('../../data/final_contact_matrices/BASE.csv')
    region_data_matrix = pd.read_csv(path, header=None, dtype=np.float64)
    regional = em.RegionMatrix(r, age_groups, region_data_matrix)
    #region_data_matrix_fix.iloc[:, 5] = region_data_matrix_fix.iloc[:, 5] * 2
    weeks_matrices_region.append(regional)

matrices_region.append(weeks_matrices_region)

contacts = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
matrices_contact = [contacts]

# Matrices contact
time_changes_contact = [1]
time_changes_region = [1]

### Set the parameters and initial conditions of the model and bundle everything together to simulate

In [3]:
# Instantiate model
model = em.PheSEIRModel()

# Set the region names, contact and regional data of the model
model.set_regions(regions)
model.read_contact_data(matrices_contact, time_changes_contact)
model.read_regional_data(matrices_region, time_changes_region)

# Initial number of susceptibles
susceptibles_array = np.array([
    [68124, 299908, 773741, 668994, 1554740, 1632059, 660187, 578319],
    [117840, 488164, 1140597, 1033029, 3050671, 2050173, 586472, 495043],
    [116401, 508081, 1321675, 1319046, 2689334, 2765974, 1106091, 943363],
    [85845, 374034, 978659, 1005275, 2036049, 2128261, 857595, 707190],
    [81258, 348379, 894662, 871907, 1864807, 1905072, 750263, 624848],
    [95825, 424854, 1141632, 1044242, 2257437, 2424929, 946459, 844757],
    [53565, 237359, 641486, 635602, 1304264, 1499291, 668999, 584130]])

susceptibles = [
    susceptibles_array[1,:].tolist(),
    np.sum(np.delete(susceptibles_array, [1], axis=0), axis=0).tolist()]

#infectives1 = (10 * np.ones((len(regions), len(age_groups)))).tolist()
#infectives1 = (1 * np.ones((len(regions), len(age_groups)))).tolist()
infectives1_array = np.array([
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 2, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0]])

infectives1 = [
    infectives1_array[1,:].tolist(),
    np.sum(np.delete(infectives1_array, [1], axis=0), axis=0).tolist()]

infectives2 = np.zeros((len(regions), len(age_groups))).tolist()

dI = 4
dL = 4

# Initial R number by region
#psis = gamma.rvs(31.36, scale=1/224, size=len(regions))
psis = (31.36/224)*np.ones(len(regions))
initial_r = np.multiply(dI*psis, np.divide(np.square((dL/2)*psis+1), 1 - 1/np.square((dI/2)*psis+1)))

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, total_days+1, 1).tolist()

# Simulate for all regions
output_scipy_solver = []
new_infec_scipy_solver = []

for r, reg in enumerate(regions):
    # List of common initial conditions and parameters that characterise the fixed and variable model
    parameters = [
        initial_r, r+1, susceptibles, np.zeros((len(regions), len(age_groups))).tolist(), np.zeros((len(regions), len(age_groups))).tolist(),
        infectives1, infectives2, np.zeros((len(regions), len(age_groups))).tolist(), np.ones((len(regions), len(times))).tolist(), dL, dI, 0.5]

    # Simulate using the ODE solver from scipy
    scipy_method = 'RK45'
    parameters.append(scipy_method)

    output_scipy_solver.append(model.simulate(list(deepflatten(parameters, ignore=str)), times))
    new_infec_scipy_solver.append(model.new_infections(output_scipy_solver[-1]))

## Figure 2

### Estimated and projected COVID-19 infections and deaths by efficacy of social restriction measures

Use different multiplier for the contact changes

In [4]:
td_mean = 15.0
td_var = 12.1**2
theta = td_var / td_mean
k = td_mean / theta
time_to_death = gamma(k, scale=theta).pdf(np.arange(1, 31)).tolist()

# Set information
fatality_ratio = (1/100 * np.array([0.0016, 0.0016, 0.0043, 0.019, 0.08975, 0.815, 3.1, 6.05])).tolist()
time_to_death.extend([0.0] * (len(times)-30))
niu = float(gamma.rvs(1, scale=1/0.2, size=1))

tests = [np.array([[1000] * len(age_groups)] * len(times))] * len(regions)
sens = 0.7
spec = 0.95

In [5]:
num_iterations = 10

# Values of reduction in contacts
reduction = [0.24, 0.48, 0.84]

deaths_mean = []
deaths_upper = []
deaths_lower = []
new_infec_scipy_solver = []

for red in reduction:
    red_deaths_mean = []
    red_deaths_upper = []
    red_deaths_lower = []
    red_new_infec_scipy_solver = []
    multiplier = 1-red

    # Reduction in contacts is applied from 23rd of March (day 37)
    betas = np.ones((len(regions), 37)).tolist()
    betas.extend((multiplier * np.ones((len(regions), len(times) - 37))).tolist())

    for r, reg in enumerate(regions):
        reg_deaths_data = np.empty((num_iterations, len(times)))
        for iter in range(num_iterations):
            # List of common initial conditions and parameters that characterise the fixed and variable model
            parameters = [
                initial_r, r+1, susceptibles, np.zeros((len(regions), len(age_groups))).tolist(), np.zeros((len(regions), len(age_groups))).tolist(),
                infectives1, infectives2, np.zeros((len(regions), len(age_groups))).tolist(), betas, dL, dI, 0.5]

            # Simulate using the ODE solver from scipy
            scipy_method = 'RK45'
            parameters.append(scipy_method)

            red_new_infec_scipy_solver.append(model.new_infections(model.simulate(list(deepflatten(parameters, ignore=str)), times)))

            # Sample death results data for all regions
            for t, _ in enumerate(times):
                reg_deaths_data[iter, t] = np.sum(model.mean_deaths(fatality_ratio, time_to_death, t, red_new_infec_scipy_solver[-1]))
            
        red_deaths_mean.append(np.mean(reg_deaths_data, axis=0))
        red_deaths_upper.append(np.quantile(reg_deaths_data, 0.975, axis=0))
        red_deaths_lower.append(np.quantile(reg_deaths_data, 0.025, axis=0))

    deaths_mean.append(red_deaths_mean)
    deaths_upper.append(red_deaths_upper)
    deaths_lower.append(red_deaths_lower)
    new_infec_scipy_solver.append(red_new_infec_scipy_solver)


In [33]:
len(times)

90

In [15]:
np.sum(new_infec_scipy_solver[0][0], axis=1).shape

(90,)

In [35]:
from plotly.subplots import make_subplots

colours = ['red', 'blue']
trace_name = regions

fig = go.Figure()
fig = make_subplots(rows=2, cols=len(reduction), horizontal_spacing = 0.15)

# Plot (bar chart cases each day)
for Reg, region in enumerate(regions):
    for r, red in enumerate(reduction):
        if r != 0:
            # Plot of incidences

            fig.add_trace(
                go.Scatter(
                    x=times,
                    y=deaths_mean[r][Reg],
                    mode='lines',
                    name=trace_name[Reg],
                    showlegend=False,
                    line_color=colours[Reg],
                    line={'dash': 'dot'}
                ),
                row= 2,
                col= r+1
            )

            fig.add_trace(
                go.Scatter(
                    x=times + times[::-1],
                    y=deaths_upper[r][Reg] + deaths_lower[r][Reg][::-1],
                    fill='toself',
                    fillcolor=colours[Reg],
                    line_color=colours[Reg],
                    opacity=0.15,
                    mode='lines',
                    showlegend=False,
                    name=trace_name[Reg]
                ),
                row= 2,
                col= r+1
            )

            fig.update_xaxes(
                row=2, col=r+1,
                ticks='inside', tickcolor='black', ticklen=5)
            fig.update_yaxes(ticks='outside', tickcolor='black', ticklen=7.5)
        
        else:

            fig.add_trace(
                go.Scatter(
                    x=times,
                    y=deaths_mean[r][Reg],
                    mode='lines',
                    name=trace_name[Reg],
                    line_color=colours[Reg],
                    line={'dash': 'dot'}
                ),
                row= 2,
                col= r+1
            )

            fig.add_trace(
                go.Scatter(
                    x=times + times[::-1],
                    y=deaths_upper[r][Reg] + deaths_lower[r][Reg][::-1],
                    fill='toself',
                    fillcolor=colours[Reg],
                    line_color=colours[Reg],
                    opacity=0.15,
                    mode='lines',
                    showlegend=False,
                    name=trace_name[r]
                ),
                row= 2,
                col= r+1
            )

            fig.update_xaxes(
                row=2, col=r+1, title_text='Deaths<br>{} reduction in transmission'.format(red),
                ticks='inside', tickcolor='black', ticklen=5)
            fig.update_yaxes(ticks='outside', tickcolor='black', ticklen=7.5)
        

# Add axis labels
fig.update_layout(
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(
        linecolor='black',
        range=[0, 62],
        tickvals=np.arange(1, 62, 10).tolist(),
        ticktext=['Mar 01', 'Mar 11', 'Mar 21', 'Mar 31', 'Apr 09', 'Apr 19', 'Apr 29']),
    xaxis2=dict(linecolor='black'),
    yaxis2=dict(
        linecolor='black',
        range=[0, 62],
        tickvals=np.arange(1, 62, 10).tolist(),
        ticktext=['Mar 01', 'Mar 11', 'Mar 21', 'Mar 31', 'Apr 09', 'Apr 19', 'Apr 29']),
    xaxis3=dict(linecolor='black'),
    yaxis3=dict(
        linecolor='black',
        range=[0, 62],
        tickvals=np.arange(1, 62, 10).tolist(),
        ticktext=['Mar 01', 'Mar 11', 'Mar 21', 'Mar 31', 'Apr 09', 'Apr 19', 'Apr 29']),
    xaxis4=dict(linecolor='black'),
    yaxis4=dict(
        linecolor='black',
        range=[0, 1.2],
        tickvals=np.arange(0, 1.1, 0.2).tolist(),
        ticktext=['0', '0.2', '0.4', '0.6', '0.8', '1']),
    xaxis5=dict(linecolor='black'),
    yaxis5=dict(
        linecolor='black',
        range=[0, 1.2],
        tickvals=np.arange(0, 1.1, 0.2).tolist(),
        ticktext=['0', '0.2', '0.4', '0.6', '0.8', '1']),
    xaxis6=dict(linecolor='black'),
    yaxis6=dict(
        linecolor='black',
        range=[0, 1.2],
        tickvals=np.arange(0, 1.1, 0.2).tolist(),
        ticktext=['0', '0.2', '0.4', '0.6', '0.8', '1']),
    legend=dict(
        orientation='h',
        yanchor="bottom",
        y=1.075,
        xanchor="right",
        x=1)
    )

fig.write_image('Figure-2.pdf')
fig.show()

IndexError: list index out of range

## Figure 3

### Uncertainty in trajectories of COVID-19 infections, deaths and reproduction number using baseline contact matrices

### Define setup matrices for the PHE Model

In [ ]:
# Populate the model
total_days =  90
regions = [
    'London', 'NW']
age_groups = ['0-1', '1-5', '5-15', '15-25', '25-45', '45-65', '65-75', '75+']

weeks = list(range(1,int(np.ceil(total_days/7))+1))
matrices_region = []

### Variable
# # Initial state of the system
# for w in weeks:
#     weeks_matrices_region = []
#     for r in regions:
#         path = os.path.join('../../data/final_contact_matrices/{}_W{}.csv'.format(r, w))
#         region_data_matrix = pd.read_csv(path, header=None, dtype=np.float64)
#         #region_data_matrix_var.iloc[:, 5] = region_data_matrix_var.iloc[:, 5] * 2
#         regional = em.RegionMatrix(r, age_groups, region_data_matrix)
#         weeks_matrices_region.append(regional)
# 
#     matrices_region.append(weeks_matrices_region)
# 
# contacts = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
# matrices_contact = [contacts]
# 
# # Matrices contact
# time_changes_contact = [1]
# time_changes_region = np.arange(1, total_days+1, 7).tolist()

### Fixed
# Initial state of the system
weeks_matrices_region = []
for r in regions:
    path = os.path.join('../../data/final_contact_matrices/BASE.csv')
    region_data_matrix = pd.read_csv(path, header=None, dtype=np.float64)
    regional = em.RegionMatrix(r, age_groups, region_data_matrix)
    #region_data_matrix_fix.iloc[:, 5] = region_data_matrix_fix.iloc[:, 5] * 2
    weeks_matrices_region.append(regional)

matrices_region.append(weeks_matrices_region)

contacts = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
matrices_contact = [contacts]

# Matrices contact
time_changes_contact = [1]
time_changes_region = [1]

### Set the parameters and initial conditions of the model and bundle everything together to simulate

In [ ]:
# Instantiate model
model = em.PheSEIRModel()

# Set the region names, contact and regional data of the model
model.set_regions(regions)
model.read_contact_data(matrices_contact, time_changes_contact)
model.read_regional_data(matrices_region, time_changes_region)

# Initial number of susceptibles
susceptibles_array = np.array([
    #[68124, 299908, 773741, 668994, 1554740, 1632059, 660187, 578319],
    [117840, 488164, 1140597, 1033029, 3050671, 2050173, 586472, 495043],
    #[116401, 508081, 1321675, 1319046, 2689334, 2765974, 1106091, 943363],
    #[85845, 374034, 978659, 1005275, 2036049, 2128261, 857595, 707190],
    [81258, 348379, 894662, 871907, 1864807, 1905072, 750263, 624848]
    #[95825, 424854, 1141632, 1044242, 2257437, 2424929, 946459, 844757],
    #[53565, 237359, 641486, 635602, 1304264, 1499291, 668999, 584130]]
    )

susceptibles = [
    susceptibles_array[1,:].tolist(),
    np.sum(np.delete(susceptibles_array, [1], axis=0), axis=0).tolist()]

#infectives1 = (10 * np.ones((len(regions), len(age_groups)))).tolist()
#infectives1 = (1 * np.ones((len(regions), len(age_groups)))).tolist()
infectives1_array = np.array([
    #[0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 2, 0, 0, 0],
    #[0, 0, 0, 0, 0, 0, 1, 0],
    #[0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    #[0, 0, 0, 1, 0, 0, 0, 0],
    #[0, 0, 0, 0, 0, 1, 0, 0]]
    )

infectives1 = [
    infectives1_array[1,:].tolist(),
    np.sum(np.delete(infectives1_array, [1], axis=0), axis=0).tolist()]

infectives2 = np.zeros((len(regions), len(age_groups))).tolist()

dI = 4
dL = 4

# Initial R number by region
#psis = gamma.rvs(31.36, scale=1/224, size=len(regions))
psis = (31.36/224)*np.ones(len(regions))
initial_r = np.multiply(dI*psis, np.divide(np.square((dL/2)*psis+1), 1 - 1/np.square((dI/2)*psis+1)))

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, total_days+1, 1).tolist()

# Simulate for all regions
output_scipy_solver = []
new_infec_scipy_solver = []

for r, reg in enumerate(regions):
    # List of common initial conditions and parameters that characterise the fixed and variable model
    parameters = [
        initial_r, r+1, susceptibles, np.zeros((len(regions), len(age_groups))).tolist(), np.zeros((len(regions), len(age_groups))).tolist(),
        infectives1, infectives2, np.zeros((len(regions), len(age_groups))).tolist(), np.ones((len(regions), len(times))).tolist(), dL, dI, 0.5]

    # Simulate using the ODE solver from scipy
    scipy_method = 'RK45'
    parameters.append(scipy_method)

    output_scipy_solver.append(model.simulate(list(deepflatten(parameters, ignore=str)), times))
    new_infec_scipy_solver.append(model.new_infections(output_scipy_solver[-1]))